In [3]:
import logging
import sys

import numpy

from matplotlib import pylab

pylab.rcParams['figure.figsize'] = (8.0, 8.0)
pylab.rcParams['image.cmap'] = 'rainbow'

import matplotlib.pyplot as plt

from rascil.data_models import PolarisationFrame
from rascil.processing_components import create_blockvisibility_from_ms, \
    advise_wide_field, plot_uvcoverage, plot_visibility, weight_visibility,\
    create_image_from_visibility, show_image, export_image_to_fits

from rascil.processing_components.imaging.ng import invert_ng
from rascil.workflows.rsexecute.execution_support import rsexecute
from rascil.workflows.rsexecute.pipelines import continuum_imaging_list_rsexecute_workflow

log = logging.getLogger()
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler(sys.stdout))

mpl_logger = logging.getLogger("matplotlib")
mpl_logger.setLevel(logging.WARNING)

Read the MeasurementSet

In [4]:
bvis=create_blockvisibility_from_ms("/Users/wangfeng/work/muserdata/output/selfmodel.ms")[0]
# bvis=create_blockvisibility_from_ms("/Users/wangfeng/work/muserdata/output/selfmodel.ms",start_chan=63,end_chan=63)[0]

print(bvis)
advice=advise_wide_field(bvis, guard_band_image=0.5) #,oversampling_synthesised_beam=3)

cellsize=advice["cellsize"]
model=create_image_from_visibility(bvis, cellsize=cellsize, npixel=1024,
                                   polarisation_frame=PolarisationFrame("stokesIV"))

print(model)


create_blockvisibility_from_ms: {'type': 'Measurement Set', 'subType': '', 'readme': 'This is a MeasurementSet Table holding measurements from a Telescope\n'}
create_blockvisibility_from_ms: Reading unique fields [0], unique data descriptions [0]
create_blockvisibility_from_ms: Found 2770560 rows
create_blockvisibility_from_ms: Found 64 channels
create_blockvisibility_from_ms: Reading all 64 channels
create_blockvisibility_from_ms: Observation from 2015-11-22 04:50:31.020 to 2015-11-22 04:51:59.995
BlockVisibility:
	Source SUN
	Phasecentre: <SkyCoord (ICRS): (ra, dec) in deg
    (237.3576405, -20.05175896)>
	Number of visibility blocks: 3552
	Number of integrations: 3552
	Visibility shape: (3552, 40, 40, 64, 2)
	Number of flags: 372817920
	Number of channels: 64
	Frequency: [4.000e+08 4.250e+08 4.500e+08 4.750e+08 5.000e+08 5.250e+08 5.500e+08
 5.750e+08 6.000e+08 6.250e+08 6.500e+08 6.750e+08 7.000e+08 7.250e+08
 7.500e+08 7.750e+08 8.000e+08 8.250e+08 8.500e+08 8.750e+08 9.000e+08
 9

In [5]:
def solve_antenna_gains_itsubs_nocrossdata(gain, gwt, x, xwt, niter=30, tol=1e-8, phase_only=True, refant=0):
    """Solve for the antenna gains using full matrix expressions, but no cross hands
    x(antenna2, antenna1) = gain(antenna1) conj(gain(antenna2))
    See Appendix D, section D.1 in:
    J. P. Hamaker, "Understanding radio polarimetry - IV. The full-coherency analogue of
    scalar self-calibration: Self-alignment, dynamic range and polarimetric fidelity," Astronomy
    and Astrophysics Supplement Series, vol. 143, no. 3, pp. 515–534, May 2000.
    :param gain: gains
    :param gwt: gain weight
    :param x: Equivalent point source visibility[nants, nants, ...]
    :param xwt: Equivalent point source weight [nants, nants, ...]
    :param niter: Number of iterations
    :param tol: tolerance on solution change
    :param phase_only: Do solution for only the phase? (default True)
    :param refant: Reference antenna for phase (default=0.0)
    :return: gain [nants, ...], weight [nants, ...]
    """
    # This implementation is sub-optimal. TODO: Reimplement IQ, IV calibration
    nants, _, nchan, npol = x.shape
    assert npol == 2
    newshape = (nants, nants, nchan, 4)
    x_fill = numpy.zeros(newshape, dtype='complex')
    x_fill[..., 0] = x[..., 0]
    x_fill[..., 3] = x[..., 1]
    xwt_fill = numpy.zeros(newshape, dtype='float')
    xwt_fill[..., 0] = xwt[..., 0]
    xwt_fill[..., 3] = xwt[..., 1]
    return solve_antenna_gains_itsubs_matrix(gain, gwt, x_fill, xwt_fill, niter=niter, tol=tol, phase_only=phase_only,
                                             refant=refant)

def read_sun_disk_data():
    sundisk_file_path = "quietsundisk400_2000MHz.txt"
    sundisk_file = open(sundisk_file_path, "r")
    sundisk_dict = {}
    try:
        while True:
            linekey = sundisk_file.readline()
            linevalue = sundisk_file.readline()
            if linekey and linevalue:
                linekey = float(linekey.strip())
                value = list(map(eval, linevalue.strip().split(',')[1:]))
                npvalue = []
                for i in range(200):
                    npvalue.append(numpy.float32(value[i]))
                sundisk_dict[linekey] = numpy.array(npvalue)
            else:
                return sundisk_dict
    finally:
        sundisk_file.close()


Read sun disk model data from the file

In [6]:
sundisk_dict = read_sun_disk_data()

Generate the Sun disk image

In [9]:
# fov - minute
size = model.data.shape[2]
fov = cellsize*180/numpy.pi * size * 60
# sun disk = 32 arc minute, 
sun_radius = 16  
scale = sun_radius * 2 / fov
print(cellsize, fov,sun_radius,scale)
for freq in range(model.data.shape[0]):
    frequency=400.+freq*25
    sun_disk = sundisk_dict[frequency]
    x0 = y0 = size //2 
    for i in range(size):
        for j in range(size):
            radius = numpy.sqrt((i-size//2)**2+(j-size//2)**2)*fov/size
            if  radius <= 16:
                #print(int(round(radius/32*200,0)))
                model.data[freq,0,i,j] = sun_disk[int(round(radius/32*200))]
    model.data[freq,1,...] = model.data[freq,0,...]



1.7277286845566425e-05 60.82041555074159 16 0.5261391213827348


In [10]:
export_image_to_fits(model, "selfmodel_model.fits")

In [ ]:
from rascil.processing_components.imaging.ng import predict_ng, invert_ng

newvis = predict_ng(bvis, model)
plot_visibility([newvis], title="Predicted vis", plot_file="selfmodel_predictedvis.png")
plt.show()